##Data preparation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import os
import kerastuner as kt
from contextlib import redirect_stdout
import shutil
import locale

locale.setlocale(locale.LC_ALL, 'en_US.UTF-8') #This is needed to convert string values of type 1,999.99 
                                               #to float 1999.99

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

#Load locally stored data:

In [2]:
#Define where are the datasets

#Paths for data with market info
#path = os.getcwd()+"/exper_files/datasets/" #Here they are datasets from yahoo
path = os.getcwd()+"/exper_files/coinmarket/" #Here they are datasets from coinmarket

#Paths for data with social info
social_path = os.getcwd()+"/exper_files/social_datasets/cryptocompare/" #Define where are the datasets from cryptocompare

Create the whole dataset:

In [3]:
def dataset_creation(crypto_list, pth, social_pth):
    datasets = {} #Store all datasets here
    social_datasets = {} #Store all datasets with social media information here
    
    min_dates = {} #Store all minimum dates here
    max_dates = {} #Store all maximum dates here
    
    datasets_list = os.listdir(pth) #Find all dataset
    social_datasets_list = os.listdir(social_pth) #Find all dataset with social media information
    
    splitted_path = path.split('/')
    if splitted_path[-2] == 'coinmarket':
        type_of_file = '.txt'
    
    elif splitted_path[-2] == 'datasets':
        type_of_file = '.csv'
        
    
    social_crypto_list = [elem+'.csv' for elem in crypto_list]
    crypto_list = [elem+type_of_file for elem in crypto_list]
    
    
    for dataset in datasets_list:
        if dataset.endswith(type_of_file) and (dataset) in crypto_list:

            name = dataset.split(".")
            dataset_name = name[0]
            
            if splitted_path[-2] == 'datasets':
                
                datasets[dataset_name] = pd.read_csv(pth + dataset) 

                datasets[dataset_name]['Date'] = pd.to_datetime(datasets[dataset_name]['Date']) #Dataset from yahoo
                                                                                            #is stored ia .csv file.
                    
                datasets[dataset].rename(columns={'Adj Close':dataset+'Adj_Close'}) #Space will cause troubles
                
            
            elif splitted_path[-2] == 'coinmarket':  
                    
                lines = []
                with open(pth + dataset) as f: #Dataset from coinmarket is stored in a .txt file, so we need to read it
                                               #line by line.
                    lines = f.readlines()
                
                initial_columns = lines[0].split("\t") #Split the first line to individual words. The first line 
                                                       #contains the column names. 
                columns = []
                for col in initial_columns:
                    clear_col = col.split('*') #Some column names contains * or **, so we need to exclude them
                    clear_col = clear_col[0].split('\n') #The last element cointains a \n
                    
                    if clear_col[0] == 'Market Cap':
                        clear_col[0] = 'Market_Cap' #Space will cause troubles
                    columns.append(clear_col[0])
                
                df = pd.DataFrame(columns=columns) #Define column names
                
                
                for line in range(1, len(lines)):
                    
                    splitted_line = lines[line].split("\t") #Split each line to individual numbers
                    
                    for elem in range(len(splitted_line)):
                        clear_elem = splitted_line[elem].split('$') #All number cointains a $ (except Date) so we need 
                                                                    #throw it away.
                        
                        if len(clear_elem) == 1:
                            splitted_line[elem] = pd.to_datetime(clear_elem[0])#Tranform each Date to 'datetime' type
                            
                        else:
                            clear_elem[1] = clear_elem[1].split('\n')
                            splitted_line[elem] = locale.atof(clear_elem[1][0])
                    
                    #Append each line to the dataframe
                    row_to_append = pd.Series(splitted_line, index=columns)                    
                    df = df.append(row_to_append, ignore_index=True)
                
                #Invert the sorting of values to ascending based on 'Date'
                df = df.sort_values(by=['Date']).reset_index().drop(['index'], axis=1)
                datasets[dataset_name] = df.copy()
            
            #There are a few missing values, so let fill them with the previous value
            datasets[dataset_name].fillna(method='ffill', inplace=True)

            #Create new columns 'close_off_high' and 'volatility' in order to make predictions more accurate:
            kwards = {'close_off_high': lambda x: 2 * (x['High'] - x['Close']) / (x['High'] - x['Low']) - 1,
              'volatility': lambda x: (x['High'] - x['Low']) / (x['Open'])
              }

            datasets[dataset_name] = datasets[dataset_name].assign(**kwards)
            
            datasets[dataset_name].fillna(-1, inplace=True) #Fill possible NaN close_off_high values with -1
                                                            #Because NaN values caused by (0/0)-1
            

            first_date = pd.to_datetime(datasets[dataset_name]['Date'][0])
            min_dates[dataset_name]=first_date
            
            last_date = pd.to_datetime(datasets[dataset_name]['Date'].iloc[-1])
            max_dates[dataset_name] = last_date
            
    
    
    #Get social media information for each coin from the corresponding csv
    for dataset in social_datasets_list:
        if dataset.endswith(".csv") and (dataset) in social_crypto_list:
            
            name = dataset.split(".")
            dataset_name = name[0]

            social_datasets[dataset_name] = pd.read_csv(social_pth + dataset) 
            
            social_datasets[dataset_name]['time'] = pd.to_datetime(social_datasets[dataset_name]['time'])
            
            #Drop first column which is an unused index
            social_datasets[dataset_name] = social_datasets[dataset_name].iloc[:, 1:]
            
            #We observed that there are many zero rows at social media datasets.
            #We should find the first row which is non-zero
            res = [next(((j, i) for i, j in enumerate(social_datasets[dataset_name][col]) if j != 0), (0, 0)) 
                   for col in social_datasets[dataset_name] if col != 'time']
            
            #Get all columns except 'time'
            columns_except_time = [col for col in social_datasets[dataset_name].columns if col != 'time']
            
            #Store the index of each column with the first non-zero element
            df_res = pd.DataFrame(res, columns=['value', 'position'], index=columns_except_time)
            
            #Get the minimum of these indices
            first_non_zero_row = df_res['position'].min()

            
            first_date = pd.to_datetime(social_datasets[dataset_name]['time'][first_non_zero_row])
            if first_date >  min_dates[dataset_name]:
                 min_dates[dataset_name] = first_date
                    
            last_date = pd.to_datetime(social_datasets[dataset_name]['time'].iloc[-1])
            if last_date < max_dates[dataset_name]:
                max_dates[dataset_name] = last_date
            
    
    
    max_date = max(min_dates.values(), key=lambda v: v)
    min_date = min(max_dates.values(), key=lambda v: v)

    #Drop all the data which are prior to max_date and later to min_date
    for dataset in datasets:
        datasets[dataset] = datasets[dataset][(datasets[dataset]['Date'] >= max_date) & 
                                              (datasets[dataset]['Date'] <= min_date)]
        
    for dataset in social_datasets:
        social_datasets[dataset] = social_datasets[dataset][(social_datasets[dataset]['time'] >= max_date) & 
                                                            (social_datasets[dataset]['time'] <= min_date)]


    #Compute the average and standard deviation of 'Close' value for the last 7-days and 30-days(month): 
    for dataset in datasets:

        temp = datasets[dataset].copy()

        #Drop the first 30 days to be able to compute average and standard deviation of month for the rows of the table
        temp = temp[29:]

        temp['mean_7days_Close'] = datasets[dataset]['Close'].rolling(window=7).mean()

        temp['mean_month_Close'] = datasets[dataset]['Close'].rolling(window=30).mean()

        temp['std_7days_Close'] = datasets[dataset]['Close'].rolling(window=7).std()

        temp['std_month_Close'] = datasets[dataset]['Close'].rolling(window=30).std()

        datasets[dataset] = temp.copy()
        
    
    #Drop the first 30 days of 'social_datasets' to be aligned with 'datasets'
    for dataset in social_datasets:
            social_datasets[dataset] = social_datasets[dataset][29:]
          


    #Rename the columns and concatenate all datasets to one 
    count = 0
    for dataset in datasets:

        datasets[dataset] = datasets[dataset].rename(columns=lambda x: dataset+'_'+x)
        datasets[dataset] = datasets[dataset].rename(columns={dataset+'_Date': 'Date'})                                                   
        
        if count == 0:
            
            date_col = (datasets[dataset]['Date'].reset_index()).drop(['index'], axis=1)
            
            
        datasets[dataset] = ((datasets[dataset].drop(['Date'], axis=1)).reset_index()).drop(['index'], axis=1)    
                    
    
    for dataset in social_datasets:
        
        social_datasets[dataset] = social_datasets[dataset].rename(columns=lambda x: dataset+'_'+x)
        
        social_datasets[dataset] = ((social_datasets[dataset].drop([dataset+'_time'], axis=1)).reset_index()).drop(['index'], 
                                                                                                          axis=1) 

    
    
    #Concatenate all datasets to one 
    whole_market_dataset = pd.concat([datasets[dataset] for dataset in datasets], axis=1)
    whole_social_dataset = pd.concat([social_datasets[dataset] for dataset in social_datasets], axis=1)
    whole_dataset = pd.concat([date_col, whole_market_dataset, whole_social_dataset], axis=1)
    
   
    return whole_dataset

Split data into training, validation and test set:

In [4]:
def split_data(perc_train_set, perc_val_set, currency_data):
        
        #Compute the date to split the dataset into training and validation_test set based on 'perc_train_set'
        splt_date_train = currency_data.iloc[round(currency_data.shape[0] * perc_train_set)]['Date']
        
        #Split the dataset into trainning and validation_test set
        tr_set, val_tst_set = currency_data[currency_data['Date'] < splt_date_train], \
                         currency_data[currency_data['Date'] >= splt_date_train]
        
        #Compute the date to split the val_tst_set into validation and test set based on 'perc_val_set'
        splt_date_val = val_tst_set.iloc[round(val_tst_set.shape[0] * perc_val_set)]['Date']

        #Split the val_tst_set into validation and test set        
        val_set, tst_set = val_tst_set[val_tst_set['Date'] < splt_date_val], \
                            val_tst_set[val_tst_set['Date'] >= splt_date_val]
        
        return tr_set, val_set, tst_set, splt_date_train, splt_date_val

Normalize training, validation and test inputs and outputs with MixMaxScaler:

In [5]:
def normalize_in_out(prd_range, wind_len, tr_set, val_set, tst_set, feats, coin_targ, crypto_list):
    
    
    #Scaling

    scaler = MinMaxScaler(feature_range=(0, 1)) #Scaler for all columns
    
    tr_set = tr_set.drop(['Date'], axis=1)
    train_scaled_data = pd.DataFrame(scaler.fit_transform(tr_set.values), columns=tr_set.columns, 
                                     index=training_set.index) #Training set fit and transform

    close_scaler = MinMaxScaler(feature_range=(0, 1)) #Scaler for 'Close' column only
    close_scaler.fit((tr_set[coin_targ+'_Close'].values).reshape(-1, 1)) #Training set fit only to 'Close' values

    val_set = val_set.drop(['Date'], axis=1)
    val_scaled_data = pd.DataFrame(scaler.transform(val_set.values), columns=val_set.columns,
                                  index=val_set.index) #Validation set just transform

    tst_set = tst_set.drop(['Date'], axis=1)
    test_scaled_data = pd.DataFrame(scaler.transform(tst_set.values), columns=tst_set.columns,
                                   index=tst_set.index) #Test set just tranform

    
    all_feats = tr_set.columns #Get all features
    feats = [crypto+"_"+feat for crypto in crypto_list for feat in feats] #Get the features in the appropriate format 
                                                                          #(e.g 'Close' --> 'BTC-USD_Close')
    
    
    #Normalize training inputs
    LSTM_tr_in = []
    for i in range(len(train_scaled_data) - wind_len):
        tmp_set = train_scaled_data[i:(i + wind_len)].copy()
        
        for col in all_feats:
            if col not in feats:
                tmp_set = tmp_set.drop([col], axis=1) #Drop the feature that will not be used

        LSTM_tr_in.append(tmp_set)
    
    #Transform from DataFrame to numpy array
    LSTM_tr_in = [np.array(LSTM_tr_i) for LSTM_tr_i in LSTM_tr_in]
    LSTM_tr_in = np.array(LSTM_tr_in)
    
    
    #Normalize validation inputs
    LSTM_val_in = []
    for i in range(len(val_scaled_data) - wind_len):
        tmp_set = val_scaled_data[i:(i + wind_len)].copy()
        
        for col in all_feats:
            if col not in feats:
                tmp_set = tmp_set.drop([col], axis=1) #Drop the feature that will not be used

        LSTM_val_in.append(tmp_set)
        
    #Transform from DataFrame to numpy array
    LSTM_val_in = [np.array(LSTM_val_i) for LSTM_val_i in LSTM_val_in]
    LSTM_val_in = np.array(LSTM_val_in)
    
    
    #Normalize test inputs
    LSTM_test_in = []
    for i in range(len(test_scaled_data) - wind_len):
        tmp_set = test_scaled_data[i:(i + wind_len)].copy() 
        
        for col in all_feats:
            if col not in feats:
                tmp_set = tmp_set.drop([col], axis=1) #Drop the feature that will not be used

        LSTM_test_in.append(tmp_set)
    
    
    #Transform from DataFrame to numpy array
    LSTM_test_in = [np.array(LSTM_test_i) for LSTM_test_i in LSTM_test_in]
    LSTM_test_in = np.array(LSTM_test_in)
    
    
    #Create LSTM outputs for training. Each output is corresponded to the days which are going to be predicted (pred_range)
    LSTM_rangd_train_out = []
    for i in range(wind_len, len(train_scaled_data[coin_targ+'_Close']) - prd_range):
        LSTM_rangd_train_out.append(train_scaled_data[coin_targ+'_Close'][i:i+prd_range].values)

    LSTM_rangd_train_out = np.array(LSTM_rangd_train_out)
    
    
    #Create LSTM outputs for validation. Each output is corresponded to the days which are going to be predicted (pred_range)   
    LSTM_rangd_val_out = []
    for i in range(wind_len, len(val_scaled_data[coin_targ+'_Close']) - prd_range):
        LSTM_rangd_val_out.append(val_scaled_data[coin_targ+'_Close'][i:i+prd_range].values)
    
    LSTM_rangd_val_out = np.array(LSTM_rangd_val_out)
    
    return LSTM_rangd_train_out, LSTM_rangd_val_out, LSTM_tr_in, LSTM_val_in, LSTM_test_in, close_scaler

Define and build model:

In [14]:
########Model with CNN layers, followed by LSTM layers, followed by Dense layers #####################
from keras.layers import Conv1D
from keras.layers import MaxPool1D
from keras.layers import AveragePooling1D
from keras.layers import Conv2D
from keras.layers import MaxPool2D

from keras.layers import BatchNormalization
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM, GRU
from keras.layers import Dropout
import tensorflow as tf


def model_tuning(inputs, outputs, output_size, validation_inputs, validation_outputs, epochs, early_stop_patience, 
                 neurons, dropout, prd_range, batch_size, keras_tuner_dir, activ_func="linear", loss="mae", 
                 optimizer="adam"):
    
    def build_model(hp):
        model = Sequential()
        
        
        ###########1D CONV############
        #filters_conv1 = hp.Choice('filters_Conv1', values=[16, 32, 64])
        #kernel_size_conv1 = hp.Choice('kernel_size_Conv1', values=[2, 3, 5])
        #strides_conv1 = hp.Choice('strides_Conv1', values=[1, 2, 3])
        #model.add(Conv1D(filters=filters_conv1, kernel_size=kernel_size_conv1, strides=strides_conv1, 
        #                 activation='relu',
        #                input_shape=(inputs.shape[1], inputs.shape[2])))
       
    
        #############2D CONV##########
        #filters_conv2 = hp.Choice('filters_Conv2', values=[16, 32, 64])
        #kernel_size_conv2 = hp.Choice('kernel_size_Conv2', values=[2, 3, 5])
        #strides_conv2 = hp.Choice('strides_Conv2', values=[(1, 1), (2, 2), (3, 3)])
        #tf.keras.layers.Conv2D(filters=filters_conv2, kernel_size=kernel_size_conv2, strides=strides_conv2, 
        #                       activation='relu',
        #                       input_shape=(1, inputs.shape[1], inputs.shape[2]), data_format=channels_first)
        
        
        #moment_batchNormConv1 = hp.Float('momentum_batchNormConv1', min_value=0.09, max_value=0.99, step=0.1)
        #model.add(BatchNormalization(momentum=moment_batchNormConv1))
        
        
        ######1D POOL########
        #pool_size_maxPool1 = hp.Choice('pool_size_MaxPool1', values=[2, 3, 4])
        #model.add(MaxPool1D(pool_size=2))
        #model.add(AveragePooling1D(pool_size=2))
        
        
        #############2D POOL##########
        #model.add(MaxPool2D(pool_size=(2, 2), data_format=channels_first)

        neurGRU1 = hp.Choice('neurons_1_GRU', values=neurons)#neurLSTM1 = hp.Choice('neurons_1_LSTM', values=neurons)
        model.add(GRU(neurGRU1, input_shape=(inputs.shape[1], inputs.shape[2])))#model.add(LSTM(neurLSTM1, input_shape=(inputs.shape[1], inputs.shape[2]))) #return_sequences=True

        #dropLSTM1 = hp.Choice('dropout_LSTM_1', values=dropout)
        #model.add(Dropout(dropLSTM1))
        
        #neurLSTM2 = hp.Choice('neurons_2_LSTM', values=neurons)
        #model.add(LSTM(neurLSTM2))
        
        #dropLSTM2 = hp.Choice('dropout_LSTM_2', values=dropout)
        #model.add(Dropout(dropLSTM2))
        
        #moment_batchNormLSTM2 = hp.Float('momentum_batchNormLSTM2', min_value=0.09, max_value=0.99, step=0.1)
        #model.add(BatchNormalization(momentum=moment_batchNormLSTM2))

        neurD1 = hp.Choice('neurons_1_Dense', values=[32, 64, 128, 256])
        model.add(Dense(units=neurD1, activation='relu'))
        
        #dropDense1 = hp.Choice('dropout_Dense_1', values=dropout)
        #model.add(Dropout(dropDense1))
        
        #moment_batchNormDense1 = hp.Float('momentum_batchNormDense1', min_value=0.09, max_value=0.99, step=0.1)
        #model.add(BatchNormalization(momentum=moment_batchNormDense1))
        
        #neurD2 = hp.Choice('neurons_2_Dense', values=[32, 64, 128, 256])
        #model.add(Dense(units=neurD2, activation='relu'))
        
        #dropDense2 = hp.Choice('dropout_Dense_2', values=dropout)
        #model.add(Dropout(dropDense2))
        
        #moment_batchNormDense2 = hp.Float('momentum_batchNormDense2', min_value=0.09, max_value=0.99, step=0.1)
        #model.add(BatchNormalization(momentum=moment_batchNormDense2))
        
        model.add(Dense(units=output_size))
        model.add(Activation(activ_func))
        
        hp_learning_rate = hp.Float(name='learning_rate', min_value=0.0001, max_value=0.05, step=0.0005)
        
        hp_beta_1 = hp.Float('beta_1', min_value=0.85, max_value=0.95, step=0.01)
        
        hp_beta_2 = hp.Float('beta_2', min_value=0.98, max_value=0.999, step=0.001)
        
        hp_epsilon = hp.Float('epsilon', min_value=1e-07, max_value=1e-08, sampling='LOG')
        
        optimizer = tf.keras.optimizers.Adam(
        learning_rate=hp_learning_rate, beta_1=hp_beta_1, beta_2=hp_beta_2, epsilon=hp_epsilon)
        
        model.compile(loss=loss, optimizer=optimizer)
        return model
    
    
    
    #Create_tuner
    tuner = kt.Hyperband(build_model,
                 objective='val_loss',
                 max_epochs=epochs,
                 factor=3,
                 directory=keras_tuner_dir,
                 project_name='keras_tuner')

    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=early_stop_patience)

    tuner.search(inputs[:-prd_range], outputs, 
                 validation_data=(validation_inputs[:-prd_range], validation_outputs), 
                 epochs=epochs, batch_size=batch_size,callbacks=[stop_early],
                 use_multiprocessing=True,
                 workers=8)

    # Get the optimal hyperparameters
    best_hps=tuner.get_best_hyperparameters(num_trials=3)[0]

    
    return best_hps, tuner

Create MAE plot:

In [7]:
def create_plot(rangd_h, path_to_sav, target_cn, loss='loss'):
    
    fig, ax1 = plt.subplots(1, 1, figsize=(25, 10))

    ax1.plot(rangd_h.epoch, rangd_h.history[loss], color='b', label='Loss')
    ax1.plot(rangd_h.epoch, rangd_h.history['val_'+loss], color='orange', label='Val_loss')
    ax1.set_xticks(range(0, len(rangd_h.epoch)))
    ax1.set_title('Training Error')
    ax1.set_ylabel('Mean Absolute Error (MAE)', fontsize=12)
    ax1.set_xlabel('#Epoch', fontsize=12)
    plt.legend()
    plt.savefig(path_to_sav+"/"+target_cn+'_MAE.png')
    plt.close(fig)
    #plt.show()

Tune model and train it:

In [15]:
#Get data
crypto_list = [['BTC-USD', 'ETH-USD', 'DOGE-USD', 'USDT-USD']] #, 'DOGE-USD', 'USDT-USD','XRP-USD', 'LTC-USD', 'XLM-USD', 
                #'XMR-USD', 'BNB-USD', 'ADA-USD', 'BCH-USD', 'LINK-USD', 'ETC-USD', 'EOS-USD', 'TRX-USD',
               #'FIL-USD', 'NEO-USD', 'XTZ-USD', 'MIOTA-USD']] #Define the sets of cryptocurrencies to be tested

first_txt_flag = 0


for cryptocurrency_list in crypto_list:
    
    cryptocurrency_list_output = 'Using cryptocurrencies: '+str(cryptocurrency_list)
    print(cryptocurrency_list_output)
    data = dataset_creation(cryptocurrency_list, path, social_path)
    
    #Split data
    percent_train_set = 0.8
    percent_val_set = 0.5

    training_set, validation_set, test_set, split_date_train, split_date_valid = split_data(percent_train_set, 
                                                                                           percent_val_set, data)

    
    features_list = [['Close']]
    """[['Close'], ['Close', 'Market_Cap', 'Volume'], ['Close', 'Open', 'High'],
                     ['Close', 'close_off_high', 'volatility'],
                     ['Close', 'mean_7days_Close', 'mean_month_Close'],
                     ['Close', 'std_7days_Close', 'std_month_Close']]""" #Define the sets of features to be tested 
    
    for featurs in features_list: 
        
        new_txt_flag = 0 
        
        featurs_output = '\tUsing the features: '+str(featurs)
        print('\tUsing the features: '+str(featurs))
        
        
        #Create inputs and outputs for the model training, validation and testing
        pred_range = 5
        window_len = 10
        features = featurs
        coin_target = 'BTC-USD'
        
        

        LSTM_ranged_training_outputs, LSTM_ranged_validation_outputs, LSTM_training_inputs, LSTM_validation_inputs, LSTM_test_inputs, clse_scaler = normalize_in_out(
                                                                                                      pred_range, window_len, 
                                                                                                      training_set, 
                                                                                                      validation_set, 
                                                                                                      test_set, 
                                                                                                      features, coin_target,
                                                                                                      cryptocurrency_list)

        batch_size_list = [64] #[1, 32, 64]
        neuron_list = [20, 40, 60, 100, 128, 256, 512]
        dropout_list = [0.0, 0.1, 0.2, 0.25, 0.3, 0.4]
        early_stop_patience = 10
        epochs = 500
        shuffle = True
        verbose = 0
         
        
        for bat_s in batch_size_list:

            #Build and train model
            batch_size = bat_s
           
            
            exper_params_output = '\t\tBatch_size: '+str(batch_size)
            print(exper_params_output)
           
            
            #Path to write log files
            cur_path = os.getcwd()+'/'
            log_dir = cur_path + "logs/fit/" + str(cryptocurrency_list) + '/' + str(featurs) + '/' + 'Batch_size='+str(batch_size)
            if not os.path.exists(log_dir):
                os.makedirs(log_dir)
            
            #Path to save its checkpoints the keras_tuner because it raises an error when it is in the current working
            #directory or in a sub-folder
            keras_tuner_dir = 'C:\keras_tuner'
            if not os.path.exists(keras_tuner_dir):
                os.makedirs(keras_tuner_dir)
            
            best_hps, tuner = model_tuning(inputs=LSTM_training_inputs, outputs=LSTM_ranged_training_outputs,
                                output_size=pred_range, 
                                validation_inputs = LSTM_validation_inputs, 
                                validation_outputs = LSTM_ranged_validation_outputs,
                                epochs=epochs, early_stop_patience=early_stop_patience, neurons=neuron_list, 
                                dropout=dropout_list, prd_range=pred_range, batch_size=bat_s, 
                                           keras_tuner_dir=keras_tuner_dir)
        
            exper_outputs = "\t\tThe hyperparameter search is complete. The optimal parameter were found to be: \n" + \
                            str(tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values)
        
            
            
            # Build the model with the optimal hyperparameters and train it
            best_model = tuner.hypermodel.build(best_hps)
            mod_history = best_model.fit(LSTM_training_inputs[:-pred_range], LSTM_ranged_training_outputs, 
                                         validation_data=(LSTM_validation_inputs[:-pred_range], 
                                                          LSTM_ranged_validation_outputs), 
                                         epochs=epochs)
            
            #Get the epoch with the best validation loss
            val_loss_per_epoch = mod_history.history['val_loss']
            best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
            exper_outputs += '\n\n\t\tBest epoch: '+ str(best_epoch) + ' with validation loss: ' + str((min(val_loss_per_epoch)))
            
            #Print results to a .txt file
            if first_txt_flag == 0:
                
                if new_txt_flag == 0:
                    
                    #Create a .txt file and write the results
                    txt_log_dir = "logs/fit/" + str(cryptocurrency_list) + '/' + str(featurs) + '/'
                    f = open(txt_log_dir+"/"+coin_target+"_results.txt", "w")
                    f.write(cryptocurrency_list_output)
                    f.write("\n"+featurs_output)
                    
                    first_txt_flag += 1
                    new_txt_flag += 1
            else:
                
                if new_txt_flag == 0:
                    
                    f.close() #Close previous txt file
                    
                    #Create a .txt file and write the results
                    txt_log_dir = "logs/fit/" + str(cryptocurrency_list) + '/' + str(featurs) + '/'
                    f = open(txt_log_dir+"/"+coin_target+"_results.txt", "w")
                    f.write(cryptocurrency_list_output)
                    f.write("\n"+featurs_output)
                    
                    new_txt_flag += 1
                    
                
            
            f.write("\n"+exper_params_output)
            f.write("\n"+exper_outputs)
            
            f.write("\n\nFull keras tuner results: \n")
            with redirect_stdout(f): 
                tuner.results_summary()
            
            
            create_plot(mod_history, log_dir, coin_target)

f.close()


#Write best model summary to a txt file
with open(cur_path + 'logs/fit/modelsummary.txt', 'w') as f_sum:
    with redirect_stdout(f_sum):
        best_model.summary()
        

#Remove folder with is created by keras tuner
shutil.rmtree(keras_tuner_dir)

Trial 725 Complete [00h 00m 04s]
val_loss: 0.029117219150066376

Best val_loss So Far: 0.023726733401417732
Total elapsed time: 00h 51m 51s
INFO:tensorflow:Oracle triggered exit
Epoch 1/500
35/35 [==============================] - 2s 24ms/step - loss: 0.1633 - val_loss: 0.0288
Epoch 2/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0445 - val_loss: 0.0409
Epoch 3/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0325 - val_loss: 0.0328
Epoch 4/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0262 - val_loss: 0.0904
Epoch 5/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0383 - val_loss: 0.0295
Epoch 6/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0268 - val_loss: 0.0563
Epoch 7/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0274 - val_loss: 0.0346
Epoch 8/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0290 - val_loss: 0.0368
Epoch 9/500
35/35 [==========

35/35 [==============================] - 0s 6ms/step - loss: 0.0210 - val_loss: 0.0408
Epoch 82/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0212 - val_loss: 0.0464
Epoch 83/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0213 - val_loss: 0.0582
Epoch 84/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0226 - val_loss: 0.0368
Epoch 85/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0213 - val_loss: 0.0612
Epoch 86/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0225 - val_loss: 0.0564
Epoch 87/500
35/35 [==============================] - 0s 7ms/step - loss: 0.0244 - val_loss: 0.0460
Epoch 88/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0196 - val_loss: 0.0488
Epoch 89/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0214 - val_loss: 0.0524
Epoch 90/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0203 - val_loss: 0.0506
Epoch 91/500


35/35 [==============================] - 0s 6ms/step - loss: 0.0168 - val_loss: 0.0552
Epoch 163/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0188 - val_loss: 0.0616
Epoch 164/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0176 - val_loss: 0.0429
Epoch 165/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0175 - val_loss: 0.0446
Epoch 166/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0164 - val_loss: 0.0541
Epoch 167/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0193 - val_loss: 0.0399
Epoch 168/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0179 - val_loss: 0.0475
Epoch 169/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0169 - val_loss: 0.0721
Epoch 170/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0188 - val_loss: 0.0526
Epoch 171/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0181 - val_loss: 0.0848
Epoc

35/35 [==============================] - 0s 6ms/step - loss: 0.0171 - val_loss: 0.0566
Epoch 244/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0157 - val_loss: 0.0452
Epoch 245/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0155 - val_loss: 0.0360
Epoch 246/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0149 - val_loss: 0.0390
Epoch 247/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0153 - val_loss: 0.0436
Epoch 248/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0160 - val_loss: 0.0364
Epoch 249/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0159 - val_loss: 0.0574
Epoch 250/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0151 - val_loss: 0.0518
Epoch 251/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0146 - val_loss: 0.0578
Epoch 252/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0148 - val_loss: 0.0570
Epoc

35/35 [==============================] - 0s 6ms/step - loss: 0.0175 - val_loss: 0.0397
Epoch 325/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0170 - val_loss: 0.0499
Epoch 326/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0162 - val_loss: 0.0473
Epoch 327/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0155 - val_loss: 0.0332
Epoch 328/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0163 - val_loss: 0.0524
Epoch 329/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0168 - val_loss: 0.0476
Epoch 330/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0172 - val_loss: 0.0583
Epoch 331/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0154 - val_loss: 0.0398
Epoch 332/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0161 - val_loss: 0.0461
Epoch 333/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0160 - val_loss: 0.0509
Epoc

35/35 [==============================] - 0s 6ms/step - loss: 0.0136 - val_loss: 0.0568
Epoch 406/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0142 - val_loss: 0.0548
Epoch 407/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0144 - val_loss: 0.0534
Epoch 408/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0128 - val_loss: 0.0493
Epoch 409/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0156 - val_loss: 0.0485
Epoch 410/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0133 - val_loss: 0.0587
Epoch 411/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0136 - val_loss: 0.0554
Epoch 412/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0138 - val_loss: 0.0476
Epoch 413/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0140 - val_loss: 0.0485
Epoch 414/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0134 - val_loss: 0.0556
Epoc

35/35 [==============================] - 0s 6ms/step - loss: 0.0130 - val_loss: 0.0572
Epoch 487/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0139 - val_loss: 0.0415
Epoch 488/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0143 - val_loss: 0.0416
Epoch 489/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0130 - val_loss: 0.0399
Epoch 490/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0126 - val_loss: 0.0490
Epoch 491/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0132 - val_loss: 0.0441
Epoch 492/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0134 - val_loss: 0.0382
Epoch 493/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0123 - val_loss: 0.0454
Epoch 494/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0126 - val_loss: 0.0375
Epoch 495/500
35/35 [==============================] - 0s 6ms/step - loss: 0.0134 - val_loss: 0.0446
Epoc